In [6]:
%%file myConv.py

import torch
import numpy as np
def myConv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
  def Svertca(matrix):

    #groups exeption
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')

    #генерация bias
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)

    #padding_mode
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad3d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad3d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad3d(padding)
      matrix = pad(matrix)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad3d(padding)
      matrix = pad(matrix)

    #генерация ядра
    if type(kernel_size) == tuple:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1], kernel_size[2])
    if type(kernel_size) == int:
      filter = torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size, kernel_size)

    #filter = b
    # bias_val = bi

    spisok = []
    for l in range(out_channels):
      feature_map = np.array([]) #генерация пустой feature-map
      for k in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride):
        for i in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride): #(filter.size - 1)*dilation + 1 при delation
          for j in range (0, matrix.shape[3]-((filter.shape[4]-1)*dilation+1)+1, stride):
            summa = 0
            for c in range (in_channels//groups): #groups
              if groups>1:
                val = matrix[l*(in_channels//groups)+c][k:k+(filter.shape[2]-1)*dilation+1:dilation, i:i+(filter.shape[3]-1)*dilation+1:dilation, j:j+(filter.shape[4]-1)*dilation+1:dilation]
              else:
                val = matrix[c][k:k+(filter.shape[2]-1)*dilation+1:dilation, i:i+(filter.shape[3]-1)*dilation+1:dilation, j:j+(filter.shape[4]-1)*dilation+1:dilation]
              mini_sum = (val*filter[l][c]).sum()
              summa = summa + mini_sum
            feature_map = np.append(feature_map, float(summa + bias_val[l])) #bias

      spisok.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1, (matrix.shape[3]-((filter.shape[4]-1)*dilation+1))//stride+1))

    return np.array(spisok), torch.tensor(np.array(filter)), torch.tensor(np.array(bias_val))

  return Svertca

Writing myConv.py


In [7]:
%%file conftest.py

from pytest import fixture
import requests

@fixture
def session():
  s = requests.Session()
  s.headers.update({"User-Agent": "pytest requests"})
  s.verify = True
  yield s
  s.close()

Overwriting conftest.py


In [17]:
%%file test_http.py

import numpy as np
import torch
import sys
import os

py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor1 = torch.rand(4, 6, 6, 6)
tensor2 = torch.rand(3, 24, 28, 20)
tensor3 = torch.rand(1, 1, 1, 1)


def test_1(session):
  myFunction = myConv3d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  result, kernel, bias_val = myFunction(tensor1)
  torchFunction = torch.nn.Conv3d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
  torchFunction.weight.data = kernel
  torchFunction.bias.data = bias_val
  myResult = str(np.round(result, 2))
  torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
  assert torchResult == myResult


def test_2(session):
  myFunction2 = myConv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  result2, kernel2, bias_val2 = myFunction2(tensor2)
  torchFunction2 = torch.nn.Conv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
  torchFunction2.weight.data = kernel2
  torchFunction2.bias.data = bias_val2
  myResult = str(np.round(result2, 2))
  torchResult = str(np.round(np.array(torchFunction2(tensor2).data), 2))
  assert torchResult == myResult


def test_3(session):
  myFunction3 = myConv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  result3, kernel3, bias_val3 = myFunction3(tensor3)
  torchFunction3 = torch.nn.Conv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
  torchFunction3.weight.data = kernel3
  torchFunction3.bias.data = bias_val3
  myResult = str(np.round(result3, 2))
  torchResult = str(np.round(np.array(torchFunction3(tensor3).data), 2))
  assert torchResult == myResult



Overwriting test_http.py


In [18]:
!pytest

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.2, pluggy-1.3.0
rootdir: /content
plugins: anyio-3.7.1
collected 3 items                                                                                  

test_http.py ...                                                                             [100%]

======================================== 3 passed in 2.73s =========================================


In [42]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor1 = torch.rand(4, 6, 6, 6)

myFunction = myConv3d(in_channels=4, out_channels=2, kernel_size=5, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
result, kernel, bias_val = myFunction(tensor1)
torchFunction = torch.nn.Conv3d(in_channels=4, out_channels=2, kernel_size=5, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val

print(result)
print('===================================')
print(np.array(torchFunction(tensor1).data))

[[[[64.53336334 66.68784332]
   [63.62639999 65.40755463]]

  [[63.46946335 63.41612625]
   [60.71492386 58.53714371]]]


 [[[68.8272934  66.29068756]
   [67.02758789 66.45349884]]

  [[71.62786865 65.44200897]
   [68.41870117 65.66234589]]]]
[[[[64.5334   66.68787 ]
   [63.62642  65.40756 ]]

  [[63.469463 63.416138]
   [60.714924 58.53714 ]]]


 [[[68.827286 66.29069 ]
   [67.02759  66.45349 ]]

  [[71.62789  65.44202 ]
   [68.41869  65.66235 ]]]]


In [31]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor2 = torch.rand(3, 10, 11, 12)

myFunction2 = myConv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=3, groups=1, bias=True, padding_mode='zeros')
result2, kernel2, bias_val2 = myFunction2(tensor2)
torchFunction2 = torch.nn.Conv3d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=3, groups=1, bias=True, padding_mode='zeros')
torchFunction2.weight.data = kernel2
torchFunction2.bias.data = bias_val2

print(result2)
print('===================================')
print(np.array(torchFunction2(tensor2).data))

[[[[20.79693222 22.25090408 22.03878784 18.59212112 21.42960358
    22.34885597]
   [22.83572006 23.03974533 23.40683365 24.60747147 23.94603157
    20.30619431]
   [22.55557251 19.97583199 20.65616798 20.55960464 20.32538605
    18.11207581]
   [23.87866974 20.46404839 21.18225861 22.69393539 19.82923508
    20.87178993]
   [21.59698868 22.92676163 22.46319389 24.39571381 22.3573761
    20.32081413]]

  [[22.00375366 19.94393539 23.49243546 21.986866   23.13560677
    21.91311646]
   [18.31953621 21.12151146 22.16110611 18.57505035 20.71571159
    21.68016052]
   [21.95678139 23.01845551 19.12196541 20.61881065 23.19519806
    20.13163376]
   [22.02478409 21.70604324 22.01552963 21.99735832 21.64468956
    22.87553215]
   [20.83804321 20.44841957 20.24603271 22.77061081 21.50125694
    21.00857544]]

  [[20.44120598 19.91256714 21.48322296 22.32414627 19.32653046
    24.33836174]
   [18.42516327 22.14182281 20.21125412 20.40465164 22.15631485
    18.49917221]
   [21.31653786 20.831199

In [34]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor3 = torch.rand(1, 1, 1, 1)

myFunction3 = myConv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
result3, kernel3, bias_val3 = myFunction3(tensor3)
torchFunction3 = torch.nn.Conv3d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
torchFunction3.weight.data = kernel3
torchFunction3.bias.data = bias_val3

print(result3)
print('===================================')
print(np.array(torchFunction3(tensor3).data))

[[[[1.16362286]]]]
[[[[1.1636229]]]]


In [53]:
py_file_location = "/content/myConv.py"
sys.path.append(os.path.abspath(py_file_location))
from myConv import myConv3d

tensor4 = torch.rand(4, 5, 8, 5)

myFunction4 = myConv3d(in_channels=4, out_channels=2, kernel_size=3, stride=3, padding=1, dilation=1, groups=2, bias=True, padding_mode='zeros')
result4, kernel4, bias_val4 = myFunction4(tensor4)
torchFunction4 = torch.nn.Conv3d(in_channels=4, out_channels=2, kernel_size=3, stride=3, padding=1, dilation=1, groups=2, bias=True, padding_mode='zeros')
torchFunction4.weight.data = kernel4
torchFunction4.bias.data = bias_val4

print(result4)
print('===================================')
print(np.array(torchFunction4(tensor4).data))

[[[[ 3.24767613  5.0991478 ]
   [ 6.47021675  7.53433466]
   [ 6.87078524  6.31496382]]

  [[ 5.95551157  8.93846226]
   [ 8.28774261 13.67443752]
   [10.45772076 10.7598114 ]]]


 [[[ 4.21545887  6.47156096]
   [ 7.71272993  8.57029915]
   [ 5.80379248  7.5198164 ]]

  [[ 5.43322802  7.81936312]
   [ 8.35266495 10.47672367]
   [10.55189037 12.83864689]]]]
[[[[ 3.2476761  5.0991473]
   [ 6.470217   7.5343347]
   [ 6.870785   6.3149633]]

  [[ 5.955511   8.938462 ]
   [ 8.287742  13.6744375]
   [10.45772   10.759811 ]]]


 [[[ 4.215459   6.4715614]
   [ 7.7127295  8.570298 ]
   [ 5.803793   7.5198164]]

  [[ 5.433229   7.819363 ]
   [ 8.352666  10.476723 ]
   [10.55189   12.838646 ]]]]
